In [14]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from scipy import stats
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from datetime import datetime
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 3)
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: unset; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split



In [15]:
#loading the dataset
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_trustpilot = pd.read_csv('data/data_trustpilot_2.csv', engine='python')

display(df_trustpilot.head())



,rating,location,username,verification,repeat_reviewer_encoded,company,subject,text,answer,answered,local_date_posted,month_posted,local_hour,time_of_day,day_of_week_posted,day_type,review_time
0,5,CA,Rob Crane,0,1,Flashbay,The company rep I worked with made my…,The company rep I worked with made my transaction smooth and quick. The samples were so helpful and the item shipping was relatively very fast. I will be a repeat customer.,0,0,2024-10-22,10,21,2,1,1,0
1,5,US,Pat Anderson,1,0,Flashbay,I highly recommend using Flashbay,I highly recommend using Flashbay. Immediately after ordering I heard from them and they were attentive to every little detail. I had notebooks shipped to Alaska for a conference and they arrived in perfect condition and delivered on time! Can't say enough good things about Flashbay!,0,0,2024-10-16,10,12,3,2,1,1
2,5,CZ,Margarita Orlova,1,0,Flashbay,Great customer service,"I had the pleasure of working with Shelby Gibson on a large order that we needed to ship to different locations. Shelby was very responsive, very kind and helpful. The order process was smooth and the final products look great. Only positive feedback from my side.",0,0,2024-10-17,10,10,3,3,1,0
3,5,US,Paola Rivas,1,0,Flashbay,Outstanding Support and Attentive Service,"I had a fantastic experience with Brian Truong. He was attentive, truly understood my needs, and provided all the support I needed. Excellent customer service—thank you!",0,0,2024-10-21,10,15,3,0,1,1
4,5,CA,Fiona Mckelvey Keenan,0,1,Flashbay,My number-one go-to for computer…,"My number-one go-to for computer accessories. Rachel was superb. She responded to my every request immediately, and her communication was satisfactory enough to get all the answers I wanted. Highly recommend!",0,0,2024-10-22,10,21,2,1,1,0


In [16]:
#First check
df_trustpilot['local_date_posted'] = pd.to_datetime(df_trustpilot['local_date_posted'])
print(df_trustpilot.columns)
display(df_trustpilot.info())

Index(['rating', 'location', 'username', 'verification',
       'repeat_reviewer_encoded', 'company', 'subject', 'text', 'answer',
       'answered', 'local_date_posted', 'month_posted', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type', 'review_time'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78404 entries, 0 to 78403
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   rating                   78404 non-null  int64         
 1   location                 78404 non-null  object        
 2   username                 78401 non-null  object        
 3   verification             78404 non-null  int64         
 4   repeat_reviewer_encoded  78404 non-null  int64         
 5   company                  78404 non-null  object        
 6   subject                  78404 non-null  object        
 7   text                     78404 non-null  o

None

STAGE 1 - TRAIN/TEST SPLIT

In [17]:
# Splitting data with stratification on rating to make sure that ratings are distributed evenly across both sets due to data imbalance
train_df, test_df = train_test_split(df_trustpilot, test_size=0.2,  
    random_state=42, stratify=df_trustpilot['rating'])

print("Training set size:", len(train_df))
print("Test set size:", len(test_df))

# Verifying rating distribution
print("Rating distribution in original data:")
print(df_trustpilot['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in training set:")
print(train_df['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in test set:")
print(test_df['rating'].value_counts(normalize=True).round(3))

Training set size: 62723
Test set size: 15681

Rating distribution in original data:
rating
5    0.497
1    0.252
4    0.095
3    0.092
2    0.064
Name: proportion, dtype: float64

Rating distribution in training set:
rating
5    0.497
1    0.252
4    0.095
3    0.092
2    0.064
Name: proportion, dtype: float64

Rating distribution in test set:
rating
5    0.497
1    0.252
4    0.095
3    0.092
2    0.064
Name: proportion, dtype: float64


STAGE 2 - HANDLE OUTLIERS

In [18]:
#No outliers need to be handled

In [19]:
#Inspecting final output 

print(df_trustpilot.columns)
print(df_trustpilot.info())
print("The Trustpilot dataset has ", df_trustpilot.shape[1], "columns")
print("The Trustpilot dataset has ", df_trustpilot.shape[0], "rows")
display(df_trustpilot.head())

Index(['rating', 'location', 'username', 'verification',
       'repeat_reviewer_encoded', 'company', 'subject', 'text', 'answer',
       'answered', 'local_date_posted', 'month_posted', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type', 'review_time'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78404 entries, 0 to 78403
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   rating                   78404 non-null  int64         
 1   location                 78404 non-null  object        
 2   username                 78401 non-null  object        
 3   verification             78404 non-null  int64         
 4   repeat_reviewer_encoded  78404 non-null  int64         
 5   company                  78404 non-null  object        
 6   subject                  78404 non-null  object        
 7   text                     78404 non-null  o

,rating,location,username,verification,repeat_reviewer_encoded,company,subject,text,answer,answered,local_date_posted,month_posted,local_hour,time_of_day,day_of_week_posted,day_type,review_time
0,5,CA,Rob Crane,0,1,Flashbay,The company rep I worked with made my…,The company rep I worked with made my transaction smooth and quick. The samples were so helpful and the item shipping was relatively very fast. I will be a repeat customer.,0,0,2024-10-22,10,21,2,1,1,0
1,5,US,Pat Anderson,1,0,Flashbay,I highly recommend using Flashbay,I highly recommend using Flashbay. Immediately after ordering I heard from them and they were attentive to every little detail. I had notebooks shipped to Alaska for a conference and they arrived in perfect condition and delivered on time! Can't say enough good things about Flashbay!,0,0,2024-10-16,10,12,3,2,1,1
2,5,CZ,Margarita Orlova,1,0,Flashbay,Great customer service,"I had the pleasure of working with Shelby Gibson on a large order that we needed to ship to different locations. Shelby was very responsive, very kind and helpful. The order process was smooth and the final products look great. Only positive feedback from my side.",0,0,2024-10-17,10,10,3,3,1,0
3,5,US,Paola Rivas,1,0,Flashbay,Outstanding Support and Attentive Service,"I had a fantastic experience with Brian Truong. He was attentive, truly understood my needs, and provided all the support I needed. Excellent customer service—thank you!",0,0,2024-10-21,10,15,3,0,1,1
4,5,CA,Fiona Mckelvey Keenan,0,1,Flashbay,My number-one go-to for computer…,"My number-one go-to for computer accessories. Rachel was superb. She responded to my every request immediately, and her communication was satisfactory enough to get all the answers I wanted. Highly recommend!",0,0,2024-10-22,10,21,2,1,1,0


In [20]:
import os
os.makedirs('data', exist_ok=True)

csv_path = 'data/data_trustpilot_3.csv'
df_trustpilot.to_csv(csv_path, index=False)
print(f"Full DataFrame exported to {csv_path}")

train_path = 'data/train_trustpilot_3.csv'
test_path = 'data/test_trustpilot_3.csv'

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training set exported to {train_path}")
print(f"Test set exported to {test_path}")

Full DataFrame exported to data/data_trustpilot_3.csv
Training set exported to data/train_trustpilot_3.csv
Test set exported to data/test_trustpilot_3.csv
